# CBorg LLM Test Notebook

Test the CBorg API connection and model availability for the ECM Extractor workshop.

## Setup: Add parent directory to path

In [3]:
import sys
from pathlib import Path

# Add backend directory to path
backend_dir = Path.cwd().parent
sys.path.insert(0, str(backend_dir))

print(f"Added to path: {backend_dir}")

Added to path: /Users/hanli/Documents/GitHub/tsbyq-workshop-2025/backend


## 1. Check Environment Variables

In [4]:
import os
from dotenv import load_dotenv

# Load .env from backend directory
env_path = backend_dir / ".env"
load_dotenv(env_path)

api_key = os.getenv("CBORG_API_KEY")
base_url = os.getenv("CBORG_BASE_URL", "https://api.cborg.lbl.gov")

print(f"API Key present: {'Yes' if api_key else 'No'}")
print(f"API Key length: {len(api_key) if api_key else 0}")
print(f"Base URL: {base_url}")

API Key present: Yes
API Key length: 25
Base URL: https://api.cborg.lbl.gov


## 2. List Available Models

Check what models are available for your API key.

In [5]:
import requests

def list_available_models(api_key, base_url):
    """List all available models for the API key."""
    url = f"{base_url}/v1/models"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        print("\n=== Available Models ===")
        if "data" in data:
            for model in data["data"]:
                model_id = model.get("id", "unknown")
                print(f"  - {model_id}")
            return [m["id"] for m in data["data"]]
        else:
            print("Response:", data)
            return []
    except Exception as e:
        print(f"Error listing models: {e}")
        return []

# available_models = list_available_models(api_key, base_url)

## 3. Test CBorg LLM Client

Test the simplified CBorg client from our workshop code.

In [6]:
from ecm_extraction.llm_clients import CBorgLLMClient


client = CBorgLLMClient()
client.chat("Hi")

'Hello! 👋 How can I help you today?'

## 4. Test with Different Models

Try different models to find which one works with your API key.

In [ ]:
# Test message
test_messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Say 'Hello, CBorg API is working!' in exactly those words."}
]

# Models to test
test_models = available_models if available_models else [
    "google/gemini-1.5-flash",
    "google/gemini-flash", 
    "google/gemini-1.5-pro",
    "google/gemini-pro",
]

print("\nTesting models...\n")
working_model = None

for model_name in test_models:
    print(f"Trying: {model_name}")
    try:
        response = client.chat_completion(test_messages, model=model_name, temperature=0.0)
        print(f"  ✅ SUCCESS")
        print(f"  Response: {response[:80]}...")
        working_model = model_name
        break
    except Exception as e:
        error_str = str(e)
        if "400" in error_str or "Invalid model" in error_str:
            print(f"  ❌ Model not available")
        else:
            print(f"  ❌ Error: {error_str[:100]}")

if working_model:
    print(f"\n🎉 WORKING MODEL FOUND: {working_model}")
    print(f"\nUpdate backend/ecm_extraction/llm_clients.py line 17 to:")
    print(f'    default_model: str = "{working_model}",')
else:
    print("\n❌ No working model found. Check your API key and available models.")